# Visualization

Work in progress ...

The goal is to visualize g-code path in the jupyter notebook.

## Setup

In [ ]:
import coursierapi._
interp.repositories() ++= Seq(MavenRepository.of("https://github.com/dzufferey/my_mvn_repo/raw/master/repository"))

In [ ]:
import $ivy.`io.github.dzufferey::libgcode:0.1-SNAPSHOT`
import $ivy.`com.lihaoyi::scalatags:0.8.2`

## Declarations

The plan is to use [X3DOM](https://www.x3dom.org/) for the vizualization.
For rapid prototyping is quite good.

To make the code readable, first the tags and attributes of the subset of X3Dom which will be used.

In [ ]:
import scalatags.Text.all._
import scalatags.stylesheet._
import almond.display.{Html, Text}

object X3D {
    
    //tags
    lazy val x3d = tag("x3d")
    lazy val scene = tag("scene")
    lazy val viewpoint = tag("viewpoint")
    lazy val navigationInfo = tag("navigationinfo")
    lazy val transform = tag("transform")
    lazy val group = tag("group")
    lazy val billboard = tag("billboard")
    lazy val shape = tag("shape")
    lazy val lineSet = tag("lineset")
    lazy val indexedLineSet = tag("indexedlineset")
    lazy val indexedFaceSet = tag("indexedfaceset")
    lazy val plane = tag("plane")
    lazy val sphere = tag("sphere")
    lazy val cone = tag("cone")
    lazy val text = tag("text")
    lazy val fontStyle = tag("fontstyle")
    lazy val coordinate = tag("coordinate")
    lazy val appearance = tag("appearance")
    lazy val material = tag("material")
    lazy val depthMode = tag("depthMode")
    lazy val lineProperties = tag("lineproperties")
    
    //attributes
    lazy val use = attr("use")
    lazy val defn = attr("def")
    lazy val position = attr("position")
    lazy val orientation = attr("orientation")
    lazy val typeParams = attr("typeparams")
    lazy val rotation = attr("rotation")
    lazy val translation = attr("translation")
    lazy val size = attr("size")
    lazy val radius = attr("radius")
    lazy val primType = attr("primtype")
    lazy val subdivision = attr("subdivision")
    lazy val vertexCount = attr("vertexcount")
    lazy val coordIndex = attr("coordindex")
    lazy val index = attr("index")
    lazy val point = attr("point")
    lazy val solid = attr("solid")
    lazy val string = attr("string")
    lazy val colorPerVertex = attr("colorpervertex")
    lazy val lit = attr("lit")
    lazy val color = attr("color")
    lazy val emissiveColor = attr("emissivecolor")
    lazy val diffuseColor = attr("diffusecolor")
    lazy val specularColor = attr("specularcolor")
    lazy val ambientIntensity = attr("ambientintensity")
    lazy val lineWidthScaleFactor = attr("linewidthscalefactor")
    lazy val readOnly = attr("readonly")
    
    val defaultRotation = "1 0 0 -1.570796326795" // z axis up
    
    //stylsheet
    object Simple extends StyleSheet {
        initStyleSheet()
        val mainViewer = cls(
            backgroundColor := "rgba(128, 128, 196, 0.4)",
            borderStyle := "solid"
        )
    }
    
    def grid(_size: Int, step: Int) = {
        val grey = appearance( material( diffuseColor := "0.0 0.0 0.0", specularColor := "0.0 0.0 0.0", emissiveColor := "0.3 0.3 0.3" ) )
        val sSize = s"${2*_size} ${2*_size}"
        val sStep = s"${2*_size/step} ${2*_size/step}"
        val sBorder = s"-${_size} -${_size} 0.0 , -${_size} ${_size} 0.0 , ${_size} ${_size} 0.0, ${_size} -${_size} 0.0"
        group(id := "grid")(
            shape(
                plane(id := "innerGrid", solid := "false", size := sSize, primType := "LINES", subdivision := sStep ),
                grey
            ),
            shape(
                indexedLineSet(coordIndex := "0 1 2 3 0", colorPerVertex := "false", lit := "false")(
                    coordinate(point := sBorder)
                ),
                grey
            )
        )
    }
    
    def lineAppearance(color: String, scale: Double = 1.0) = {
        appearance(
            material( diffuseColor := "0 0 0", specularColor := "0 0 0", emissiveColor := color ),
            lineProperties( lineWidthScaleFactor := scale.toString )
        )
    }
    
    def axes(length: Int, step: Int) = {
        group(id := "axes")(
            shape( // X
                lineSet( vertexCount := "2" ) (
                    coordinate( point := s"0 0 0.001, $length 0 0.001")
                ),
                lineAppearance("1 0 0")
            ),
            shape( // Y
                lineSet( vertexCount := "2" ) (
                    coordinate( point := s"0 0 0.001, 0 $length 0.001")
                ),
                lineAppearance("0 1 0")
            ),
            shape( // Z
                lineSet( vertexCount := "2" ) (
                    coordinate( point := s"0 0 0, 0 0 $length")
                ),
                lineAppearance("0 0 1")
            ),
            for (z <- 0 to length by step) yield {
                shape(
                    lineSet( vertexCount := "2" ) (
                        coordinate( point := s"0 0 $z, $step 0 $z")
                    ),
                    lineAppearance("0 0 1")
                )
            }
        )
    }
    
    def viewer(content: Modifier) = {
        div(backgroundColor := "rgba(128, 128, 196, 0.4)",
            borderStyle := "solid",
            script( tpe := "text/javascript", src := "https://www.x3dom.org/download/x3dom.js" ),
            link( rel := "stylesheet", tpe := "text/css", href := "https://www.x3dom.org/download/x3dom.css" ),
            tag("style")(Simple.styleSheetText),
            x3d(Simple.mainViewer)(
                scene(
                    viewpoint( id:= "viewPoint", position := "5.53912 7.69774 6.54642" , orientation := "-0.69862 0.66817 0.25590 1.00294" ),
                    navigationInfo( id := "navi", tpe := "\"TURNTABLE\" \"ANY\"", typeParams := "-0.4, 60, 0.5, 1.55" ),
                    transform(rotation := defaultRotation)(
                        grid(10, 1),
                        axes(100, 1),
                        content
                    )
                )
            )
        ).render
    }
    
    //some helpers to create elements
    def display(content: Modifier) =  Html(viewer(content))
    
}

A simple test

In [ ]:
X3D.display(
    X3D.shape(
        X3D.sphere(X3D.radius := "0.2"),
        X3D.material( X3D.emissiveColor := "0.8 0.8 0.8" )
    )
)

Now that there is a basic viewer, we need to turn "plot" g-code.
For that the simplest is to extend `AbstractMachine` so that it produces lines for the different motions.

In [ ]:
import libgcode.abstractmachine._
import Plane._
import scala.collection.mutable.ArrayBuffer
import scala.collection.mutable.StringBuilder

class Tracer extends AbstractMachine {
    // TODO ...
    
    import X3D._
    
    var circularInterpolationPrecision = 0.5 //roughly one line segment every 0.5 mm
    
    protected val toDraw = ArrayBuffer.empty[Modifier]
     
    def result = toDraw.toSeq
    
    override protected def linearMotion(x: Double, y: Double, z: Double,
                                        a: Double, b: Double, c: Double,
                                        f: Double) = {
        val x0 = getX
        val y0 = getY
        val z0 = getZ
        super.linearMotion(x,y,z,a,b,c,f)
        val x1 = getX
        val y1 = getY
        val z1 = getZ
        val points = coordinate( point := s"${x0} ${y0} ${z0}, ${x1} ${y1} ${z1}")
        val appear = if (f == maxFeed) lineAppearance("0.5 0.5 0.5") else lineAppearance("0 0 0")
        val line = shape(lineSet( vertexCount := "2" )( points ), appear )
        toDraw += line
    }
    
    override protected def circularMotion(_x: Double,_y: Double,_z: Double, // end position
                                                        a: Double, b: Double, c: Double, // end orientation
                                                        _i: Double,_j: Double,_k: Double, // center of rotation
                                                        clockwise: Boolean, p: Int, f: Double) = { // number of turns and feedrate
        val coeff = if (useMillimeters) 1 else 25.4
        val x = coeff * _x
        val y = coeff * _y
        val z = coeff * _z
        val i = coeff * _i
        val j = coeff * _j
        val k = coeff * _k
        // get the center
        val cx = this.x + i
        val cy = this.y + j
        val cz = this.z + k
        val (radius, angle, offset, pitchOver2Pi) = plane match {
            case XY =>
                val radius = math.hypot(i, j)
                val angle = getRotationAngle(cx, cy, this.x, this.y, x, y, clockwise, p)
                val offset = math.atan2(y - cy, x - cx)
                val pitchOver2Pi = (z - this.z) / angle
                (radius, angle, offset, pitchOver2Pi)
            case ZX => 
                val radius = math.hypot(k, i)
                val angle = getRotationAngle(cz, cx, this.z, this.x, z, x, clockwise, p)
                val offset = math.atan2(x - cx, z - cz)
                val pitchOver2Pi = (y - this.y) / angle
                (radius, angle, offset, pitchOver2Pi)
            case YZ => 
                val radius = math.hypot(j, k)
                val angle = getRotationAngle(cy, cz, this.y, this.z, y, z, clockwise, p)
                val offset = math.atan2(z - cz, y - cy)
                val pitchOver2Pi = (x - this.x) / angle
                (radius, angle, offset, pitchOver2Pi)
        }
        val delta0 = radius / 2 / math.Pi / circularInterpolationPrecision
        val delta = if (clockwise) -delta0 else delta0
        assert(math.signum(delta) == math.signum(angle))
        def pointAt(a: Double) = {
            plane match {
                case XY => (cx + radius * math.cos(a+offset), cy + radius * math.sin(a+offset), this.z + pitchOver2Pi * a)
                case ZX => (cx + radius * math.sin(a+offset), this.y + pitchOver2Pi * a, cz + radius * math.cos(a+offset))
                case YZ => (this.x + pitchOver2Pi * a, cy + radius * math.cos(a+offset), cz + radius * math.sin(a+offset))
            }
        }
        var count = 1
        val points = new StringBuilder()
        var a = 0.0
        val (x0, y0, z0) = pointAt(a)
        points += s"${x0} ${y0} ${z0}"
        while ((a - angle).abs > delta + 1e-5) {
            val a1 = a + delta
            count += 1
            val (x1, y1, z1) = pointAt(a1)
            val points = coordinate( point := s", ${x1} ${y1} ${z1}")
            count += 1
            a = a1
        }
        val line = shape(lineSet( vertexCount := count.toString )( points.toString ), lineAppearance("0 0 0") )
        toDraw += line
        super.circularMotion(_x,_y,_z,a,b,c,_i,_j,_k,clockwise,p,f)
    }
    
}